# Query Construction 
- Có thể thấy, pipeline thông thường của mô hình RAG dữ liệu được lấy từ các vectorstore, trích xuất thông tin dựa vào mô hình embedding hoạt động chủ yếu dựa trên dữ liệu dạng phi cấu trúc. Tuy nhiên, nếu truy xuất dữ liệu có cấu trúc hay bán cấu trúc thì các mô hình RAG sẽ trích xuất thông tin như thế nào? 

<p align="center">
    <img src="../doc/image/query-construction.png" alt="basic-pipeline" width="600"/>
</p>

- Với mỗi loại dữ liệu khác nhau, cần có một module riêng cho việc chuyển đổi các đoạn query người dùng thành dạng ngôn gnuwx giúp trích xuất dữ liệu phù hợp 

# Setup 

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Text-to-metadata-filter
- Vectordb còn lưu trữ các thông tin metadata của dữ liệu. Các metadata này tồn tại dưới dạng dữ liệu có cấu trúc, để trích xuất dữ liệu này thông qua query ngươi dùng, cần phải chuyển đổi ngôn ngữ tự nhiên thành một dạng query có cấu trúc. 
- Ví dụ dưới đây sẽ trích xuất thông tin metadata của một video youtube 

In [4]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4", add_video_info=True
).load()

docs[0].metadata

{'source': 'pbAd8O1Lvm4',
 'title': 'Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'description': 'Unknown',
 'view_count': 22692,
 'thumbnail_url': 'https://i.ytimg.com/vi/pbAd8O1Lvm4/hq720.jpg',
 'publish_date': '2024-02-07 00:00:00',
 'length': 1058,
 'author': 'LangChain'}

In [5]:
import datetime
from typing import Literal, Optional, Tuple 
from langchain_core.pydantic_v1 import BaseModel, Field

# setup query schema
# thông tin trích xuất ra có cấu trúc 
class YoutubeMetadataSearch(BaseModel): 

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI 

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash-001', temperature = 0, api_key=GOOGLE_API_KEY)
structured_llm = llm.with_structured_output(YoutubeMetadataSearch)

query_transform = prompt | structured_llm

# thử nghiệm 
query_transform.invoke(
    {
        "question": "how to use multi-modal models in an agent, only videos under 5 minutes"
    }
).pretty_print()

content_search: multi-modal models agent
title_search: multi-modal agent
max_length_sec: 300


# Text-to-SQL
- Để trích xuất dữ liệu từ một cơ sở dữ liệu quan hệ, các câu hỏi của người dùng cần được chuyển đổi thành query dưới dạng SQL để trích xuất thông tin từ cơ sở dữ liệu này. 

<p align="center">
    <img src="../doc/image/text-to-sql.png" alt="basic-pipeline" width="600"/>
</p>

- Dựa vào thiết kế trên, trích xuất dữ liệu thông qua 3 phần chính: 

1. Chuyển đổi câu hỏi thành các câu query 
2. Trích xuất dữ liệu từ câu query 
3. Trả về kết quả cho người dùng.
- Trong phần này sẽ minh họa cách trích xuất thông tin từ cơ sở dữ liệu Chinook 

In [2]:
## SETUP ## 

# Chuẩn bị cơ sở dữ liệu 
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/Chinook.db")
db.run("SELECT * FROM Artist LIMIT 10;")


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [3]:
# chuyển đổi câu query thành sql 
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash-001', temperature = 0, api_key=GOOGLE_API_KEY)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SQLQuery: SELECT COUNT(*) FROM Employee'

In [4]:

db.run('SELECT COUNT(*) FROM Employee')

'[(8,)]'

In [5]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [7]:
# Trích xuất thông tin từ câu query 
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda

def remove_input(input): 
    return input.split("SQLQuery: ")[1]

execute_query = QuerySQLDataBaseTool(db = db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | RunnableLambda(remove_input) | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

In [9]:
# trả lời câu hỏi từ người dùng
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | RunnableLambda(remove_input)| execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 8 employees. \n'